![UoE](./docs/dc.jpg)

# DNAseq Coursework - Processing and analysis of high throughput DNA sequencing data

KS Singh<sup>1\*</sup>

<sup>1</sup>College of Life and Environmental Sciences;
Penryn Campus, University of Exeter TR10 9FE Cornwall UK

<sup>\*</sup> Correspondence: <ks575@exeter.ac.uk>


## Abstract

The latest technological advancement along this line, namely next generation of sequencing (NGS), allows to routinely sequence and re-sequence the whole genome of single individuals in a single laboratory within a couple of weeks and at comparably low cost. Having a tool for sequencing massive amounts of DNA enables us to investigate almost any question that is associated with the genetic sequence. First, it allows us to determine the nucleotide sequence of a target region (e.g., all exonic regions or the whole exome) or the complete genome and to identify known as well as novel single nucleotide polymorphisms (SNPs) in the sequenced region. Furthermore, paired reads facilitate the investigation of larger structural variants such as inversions, deletions, and insertions. Analysis of variants reveals the genetic makeup of that particular species and also accounts for differences from other organism or differences due to different conditions. This coursework summarizes the main approaches to analyzing DNAseq data using GATK pipeline and Samtools. It demonstrates approaches to map short-read paired-end sequencing data against the reference genome and calling high quality variants from the sample(s) of interest. 

1. [Introduction]()
2. [Know your data]()
3. [Variant calling approaches]()
4. [File formats]()
5. [Short-read mapping]()
6. [Variant calling]()
7. [Variant filtering]()
8. [Variant annotation using SnpEff]()
9. [Process VCF using Python]()


## Introduction

### General Terminologies

**Variant** 
By genetic variant we mean differences between your sample reads and a "reference" genome. As an example, imagine we are sequencing a "sample". Here "sample" can mean anything that you are interested in studying, from a cell culture, to a mouse or a cancer patient. It is a standard procedure to compare your sample sequences against the corresponding "reference genome". For instance you may compare the cancer patient genome against the "reference genome". In a typical sequencing experiment, you will find many places in the genome where your sample differs from the reference genome. These are called "genomic variants" or just "variants". 

Typically, variants are categorized as follows*:

|Type|Meaning|Example|
| --- | --- | --- |
| SNP | Single-Nucleotide Polymorphism | Reference = A; Sample = C |
| INS | Insertion | Reference = A; Sample = AGT |
| DEL | Deletion | Reference = AC; Sample = C |
| MNP | Multiple-Nucleotide Polymorphism | Reference = ATA; Sample = GTC |
| MIXED | Multiple Nucleotide & Insertion Deletion | Reference = ATA; Sample = GTCAGT |   

*It’s not a comprehensive list but just to give you an idea

**Haplotype**
A haplotype is a set of DNA variations, or polymorphisms, that tend to be inherited together. A haplotype can refer to a combination of alleles or to a set of single nucleotide polymorphisms (SNPs) found on the same chromosome.

**SNP Calling** 
process of identifying variable sites.
Genotype calling: process that determines the genotype for each individual at each site.

![gatk](gatk.png)


## know your data

- Calling variants from high coverage DNAseq data
- Calling variants from low/shallow coverage DNAseq data
- Calling variants from RNAseq data
- Calling variants from Linkage mapping data (RADseq)

## Variant calling approaches

### GATK

![gatk_workflow](./docs/gatk_workflow.png)

## File formats

### BAM format

### VCF format

In [1]:
%%bash
#check the file structure
ls -la
pwd
seqtk
seqtk sample

total 60
drwxr-xr-x 11 ks575 domain^users  4096 May 30 15:50 .
drwxr-xr-x  5 ks575 domain^users    63 May 29 09:05 ..
drwxr-xr-x  2 ks575 domain^users  4096 May 28 18:14 Bay-S
-rw-r--r--  1 ks575 domain^users 19672 May 30 15:50 dnaseq.ipynb
drwxr-xr-x  3 ks575 domain^users  4096 May 30 15:48 docs
drwxr-xr-x  2 ks575 domain^users  4096 May 28 18:15 genome
-rw-r--r--  1 ks575 domain^users  9870 May 29 01:02 hist.txt
drwxr-xr-x  2 ks575 domain^users    44 May 28 22:45 .ipynb_checkpoints
drwxr-xr-x  2 ks575 domain^users  4096 May 28 18:14 Nl33
drwxr-xr-x  2 ks575 domain^users  4096 May 28 18:14 Nl55
drwxr-xr-x  2 ks575 domain^users    67 May 28 18:16 snpeff
drwxr-xr-x  3 ks575 domain^users    28 May 28 18:15 snpeff_data
drwxr-xr-x  2 ks575 domain^users  4096 May 28 18:14 VCF
/home/ISAD/ks575/Data/BUF19/DNAseq



Usage:   seqtk <command> <arguments>
Version: 1.3-r106

Command: seq       common transformation of FASTA/Q
         comp      get the nucleotide composition of FASTA/Q
         sample    subsample sequences
         subseq    extract subsequences from FASTA/Q
         fqchk     fastq QC (base/quality summary)
         mergepe   interleave two PE FASTA/Q files
         trimfq    trim FASTQ using the Phred algorithm

         hety      regional heterozygosity
         gc        identify high- or low-GC regions
         mutfa     point mutate FASTA at specified positions
         mergefa   merge two FASTA/Q files
         famask    apply a X-coded FASTA to a source FASTA
         dropse    drop unpaired from interleaved PE FASTA/Q
         rename    rename sequence names
         randbase  choose a random base from hets
         cutN      cut sequence at long N
         listhet   extract the position of each het


Usage:   seqtk sample [-2] [-s seed=11] <in.fa> <frac>|<number>

Options:

In [ ]:
%%bash
dnaseq=$(pwd)
#Subsample the raw DNAseq data (NOTE:this step is not required when you will analyze your own datasets)
#First interleave (merge R1 and R2 together in one file) the two fastq file 
#Bay-S
seqtk mergepe All_Bay-S_R1.fastq All_Bay-S_R2.fastq > All_Bay-S.fastq
seqtk sample -2 -s 18 All_Bay-S.fastq 100000 > $dnaseq/Bay-S/All_Bay-S_seed_18_100k.fastq
#Nl33
seqtk mergepe All_Nl33-eth_R1.fastq All_Nl33-eth_R2.fastq > All_Nl33-eth.fastq
seqtk sample -2 -s 18 All_Nl33-eth.fastq 100000 > $dnaseq/Nl33/All_Nl33-eth_seed_18_100k.fastq
#Nl55
seqtk mergepe All_Nl55-eth_R1.fastq All_Nl55-eth_R2.fastq > All_Nl55-eth.fastq
seqtk sample -2 -s 18 All_Nl55-eth.fastq 100000 > $dnaseq/Nl55/All_Nl55-eth_seed_18_100k.fastq

## Short read mapping


### Index the reference genome

1. **FASTA index** We use the faidx command in samtools to prepare the fasta index file. This file describes byte offsets in the fasta file for each contig, allowing us to compute exactly where a particular reference base at contig:pos is in the fasta file. This creates a file called reference.fa.fai, with one record per line for each of the contigs in the FASTA reference file. Each record is composed of the contig name, size, location, basesPerLine and bytesPerLine.

2. **FASTA dictionary** This creates a file called reference.dict formatted like a SAM header, describing the contents of your reference FASTA file.The GATK uses two files to access and safety check access to the reference files: a .dict dictionary of the contig names and sizes and a .fai fasta index file to allow efficient random access to the reference bases. You have to generate these files in order to be able to use a Fasta file as reference.

3. **BWA index** This creates a set of files called BWA index files to get an efficient fast random access of short reads to reference genome.

In [ ]:
%%bash
dnaseq=$(path)
#generate fasta index using samtools faidx function
samtools faidx $dnaseq/genome/genome.fa
#generate fasta dictionary using picard's createSequenceDictionary.jar
picard CreateSequenceDictionary.jar R=$dnaseq/genome/genome.fa O=$dnaseq/genome/genome.dict
#generate bwa index
bwa index -p $dnaseq/genome/nillu $dnaseq/genome/genome.fa

In [7]:
%%bash
#Check the index files and their structure
ls -la ./genome/

total 3094940
drwxr-xr-x  2 ks575 domain^users       4096 May 28 18:15 .
drwxr-xr-x 11 ks575 domain^users       4096 May 28 23:17 ..
-rw-r--r--  1 ks575 domain^users    5713826 May 28 18:14 genome.dict
-rw-r--r--  1 ks575 domain^users 1159219512 May 28 18:15 genome.fa
-rw-r--r--  1 ks575 domain^users    1659854 May 28 18:15 genome.fa.fai
-rw-r--r--  1 ks575 domain^users    1205884 May 28 18:15 nillu.amb
-rw-r--r--  1 ks575 domain^users    5014274 May 28 18:15 nillu.ann
-rw-r--r--  1 ks575 domain^users 1140786412 May 28 18:15 nillu.bwt
-rw-r--r--  1 ks575 domain^users  285196579 May 28 18:15 nillu.pac
-rw-r--r--  1 ks575 domain^users  570393208 May 28 18:15 nillu.sa


In [ ]:
%%bash
#Map the short reads
dnaseq=$(pwd)
bwa mem -M -t 60 $dnaseq/genome/nillu $dnaseq/Bay-S/All_Bay-S_seed_18_100k.fastq | samtools view -b -S -F 4 -o $dnaseq/Bay-S/All_Bay-S_seed_18_100k.bam -
bwa mem -M -t 60 $dnaseq/genome/nillu $dnaseq/Nl33/All_Nl33-eth_seed_18_100k.fastq | samtools view -b -S -F 4 -o $dnaseq/Nl33/All_Nl33-eth_seed_18_100k.bam -
bwa mem -M -t 60 $dnaseq/genome/nillu $dnaseq/Nl55/All_Nl55-eth_seed_18_100k.fastq | samtools view -b -S -F 4 -o $dnaseq/Nl55/All_Nl55-eth_seed_18_100k.bam -

In [ ]:
%%bash
dnaseq=$(pwd)
#Sort the bam files
samtools sort $dnaseq/Bay-S/All_Bay-S_seed_18_100k.bam -o $dnaseq/Bay-S/All_Bay-S_seed_18_100k.sorted.bam
samtools sort $dnaseq/Nl33/All_Nl33-eth_seed_18_100k.bam -o $dnaseq/Nl33/All_Nl33-eth_seed_18_100k.sorted.bam
samtools sort $dnaseq/Nl55/All_Nl55-eth_seed_18_100k.bam -o $dnaseq/Nl55/All_Nl55-eth_seed_18_100k.sorted.bam

In [ ]:
%%bash
dnaseq=$(pwd)
#Add or replace read groups in bam files
picard AddOrReplaceReadGroups I=$dnaseq/Bay-S/All_Bay-S_seed_18_100k.sorted.bam O=$dnaseq/Bay-S/All_Bay-S_seed_18_100k.sorted.RG.bam RGID=1 RGLB=Bay-S RGPL=illumina RGPU=unit1 RGSM=Bay-S
picard AddOrReplaceReadGroups I=$dnaseq/Nl33/All_Nl33-eth_seed_18_100k.sorted.bam O=$dnaseq/Nl33/All_Nl33-eth_seed_18_100k.sorted.RG.bam RGID=2 RGLB=Nl33-eth RGPL=illumina RGPU=unit1 RGSM=Nl33-eth
picard AddOrReplaceReadGroups I=$dnaseq/Nl33/All_Nl55-eth_seed_18_100k.sorted.bam O=$dnaseq/Nl33/All_Nl55-eth_seed_18_100k.sorted.RG.bam RGID=3 RGLB=Nl55-eth RGPL=illumina RGPU=unit1 RGSM=Nl55-eth

In [ ]:
%%bash
dnaseq=$(pwd)
#Mark duplicates in bam files
picard MarkDuplicates I=$dnaseq/Bay-S/All_Bay-S_seed_18_100k.sorted.RG.bam O=$dnaseq/Bay-S/All_Bay-S_seed_18_100k.sorted.RG.MD.bam M=marked_dup_metrics.txt
picard MarkDuplicates I=$dnaseq/Bay-S/All_Nl33-eth_seed_18_100k.sorted.RG.bam O=$dnaseq/Bay-S/All_Nl33-eth_seed_18_100k.sorted.RG.MD.bam M=marked_dup_metrics.NL33.txt
picard MarkDuplicates I=$dnaseq/Bay-S/All_Nl55-eth_seed_18_100k.sorted.RG.bam O=$dnaseq/Bay-S/All_Nl55-eth_seed_18_100k.sorted.RG.MD.bam M=marked_dup_metrics.NL55.txt


In [ ]:
%%bash
dnaseq=$(pwd)
#Index the bam files
samtools index $dnaseq/Bay-S/All_Bay-S_seed_18_100k.sorted.RG.MD.bam
samtools index $dnaseq/Nl33/All_Nl33-eth_seed_18_100k.sorted.RG.MD.bam
samtools index $dnaseq/Nl55/All_Nl55-eth_seed_18_100k.sorted.RG.MD.bam

#### Base (Quality Score) Recalibration
#### Tools: BaseRecalibrator, Apply Recalibration, AnalyzeCovariates (optional)

java -Xmx10g -jar GenomeAnalysisTK.jar 
-T BaseRecalibrator 
-I LIB_sorted_RG_dup.bam  
-R ref.fa 
--knownSites LIB_qual_filtered.vcf 
-o recalibration_report.grp


java -Xmx10g -jar GenomeAnalysisTK.jar 
-T PrintReads -R ref.fa 
-I LIB_sorted_RG_dup.bam 
-BQSR recalibration_report.grp 
-o LIB_sorted_RG_dup_BQSR.bam

**NOTE**
*To perform base quality score recalibration, a reference SNP call file is required. Normally we don’t have it. If you still want to perform BQSR then skip this step and follow the GATK best practices and generate a high quality SNP calls. Then return to BQSR step and run BaseRecalibrator with SNP calls generated in 1st Pass and redo other steps following BQSR*.

![gatk_workflow](./docs/gatk_workflow.png)

In [ ]:
%%bash
dnaseq=$(pwd)
#Call variants using haplotype caller
gatk -T HaplotypeCaller -R $dnaseq/genome/genome.fa -I $dnaseq/Bay-S/All_Bay-S_seed_18_100k.sorted.RG.MD.bam -ERC GVCF -variant_index_type LINEAR -variant_index_parameter 128000 -dontUseSoftClippedBases -stand_call_conf 10.0 -o $dnaseq/Bay-S/All_Bay-S_seed_18_100k.vcf
#
gatk -T HaplotypeCaller -R $dnaseq/genome/genome.fa -I $dnaseq/Nl33/All_Nl33-eth_seed_18_100k.sorted.RG.MD.bam -ERC GVCF -variant_index_type LINEAR -variant_index_parameter 128000 -dontUseSoftClippedBases -stand_call_conf 10.0 -o $dnaseq/Nl33/All_Nl33-eth_seed_18_100k.vcf
#
gatk -T HaplotypeCaller -R $dnaseq/genome/genome.fa -I $dnaseq/Nl55/All_Nl55-eth_seed_18_100k.sorted.RG.MD.bam -ERC GVCF -variant_index_type LINEAR -variant_index_parameter 128000 -dontUseSoftClippedBases -stand_call_conf 10.0 -o $dnaseq/Nl55/All_Nl55-eth_seed_18_100k.vcf

![GVCF](./docs/blocks.png)

In [ ]:
%%bash
dnaseq=$(pwd)
#Perform joint genotyping on the cohort
gatk -R $dnaseq/genome/genome.fa -T GenotypeGVCFs --variant $dnaseq/Bay-S/All_Bay-S_seed_18_100k.vcf --variant $dnaseq/Nl33/All_Nl33-eth_seed_18_100k.vcf --variant $dnaseq/Nl55/All_Nl55-eth_seed_18_100k.vcf -O $dnaseq/VCF/All_nillu_samples.vcf

![vcf1](./docs/vcf1.png)

In [ ]:
%%bash
dnaseq=$(pwd)
#Filter INDELS from the VCF file
gatk -R $dnaseq/genome/genome.fa -T SelectVariants -V $dnaseq/VCF/All_nillu_samples.vcf -o $dnaseq/VCF/All_nillu_samples.snps.vcf --selectTypeToExclude INDEL

![vcf2-1](./docs/vcf2-1.png)

![vcf2-2](./docs/vcf2-2.png)

In [ ]:
%%bash
dnaseq=$(pwd)
#Filter low quality SNPs calls
gatk -R $dnaseq/genome/genome.fa -T VariantFiltration -V $dnaseq/VCF/All_nillu_samples.snps.vcf --filterExpression "QD < 5.0 || FS > 60.0 || MQ < 60.0 || HaplotypeScore > 13.0 || MappingQualityRankSum < -12.5 || ReadPosRankSum < -8.0" --filterName "NO_Quality" -o $dnaseq/VCF/All_nillu_samples.snps.filtered.vcf

![vcf3](./docs/vcf3.png)

### SNP calling using *samtools*

SAMTools based approach is not recommended for RNA-Seq data. It’s good to have a brief summary of SNPs using this approach but to get more confidence on estimated SNPs, one should use GATK pipeline. 

![samtools](./docs/samtools.png)

![samtools](./docs/pileup.png)
![samtools](./docs/pileup2.png)

### Variant annotation

Download the snpEff installer from the website (http://snpeff.sourceforge.net/).

In order to produce the annotations, SnpEff requires a database. Currently, there are pre-built database for over 20,000 reference genomes. This means that most cases are covered. In some very rare occasions, people need to build a database for an organism not currently supported (e.g. the genome is not publicly available). 

Which databases are supported? You can find out all the supported databases by running the database command:
 
    java -jar snpEff.jar databases | less

This command shows the database name, genome name and source data (where was the genome reference data obtained 
from). 

**Create Nilaparvata lugens database**
1. Edit the config file.
2. Nlugens.genome : Nilaparvata_lugens
3. Edit the data.dir attribute in config file
4. rename the gff in to genes.gff and fasta file to sequences.fa
5. Put bothe genes.gff and sequences.fa in ./Nlugens/ directory

[Example config file](https://github.com/pcingola/SnpEff/blob/master/snpEff.config)

Run the command

    java -Xmx4g -jar snpEff.jar build -gff3 v Nlugens

now run the following command for snpeff

    java -Xmx4g -jar /home/data/kumars/Additional_tools/snpEff/snpEff.jar Nlugens /home/data/kumars/Helicoverpa_armigera/GenotypeGCVFs/all/all_merge_AY_DF_CR.vcf > test_ann_1st_run.vcf

[SNP Annotation](./snpeff/snpEff_summary.html)